In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import log
import ultranest
from ultranest.plot import cornerplot
from scipy.stats import poisson
import pandas as pd
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
import astropy.units as u
from matplotlib import colors as mcolors
from ultranest.utils import resample_equal
from scipy.interpolate import interp1d
import scipy.stats as stats
import ultranest.stepsampler
import itertools
import gc
import json
import time
from my_functions import *

In [2]:
# Priors and likelihood functions
# Monopole
def monopole_prior_transform(cube):
    params = cube.copy()
    params[0] = cube[0]*200 # 0 - 200 (Nbar)
    
    return params

def monopole_likelihood(params):
    N_bar = params[0]
    
    lambda_i = N_bar * np.ones(NPIX)
    
    return np.sum(poisson.logpmf(m, mu=lambda_i))

# Dipole
def dipole_prior_transform(cube):
    params = cube.copy()
    params[0] = cube[0]*200 # 0 - 100
    params[1] = cube[1]/10 # 0 - 0.1
    params[2] = cube[2]*2*np.pi # 0 - 2pi
    
    c = cube[3] # 0 - 1, used to calculate theta below. 
    # Oliver uses 0 - 0.1, but this doesnt return theta between (0,pi), is it a typo or am i missing something else?
    params[3] = np.arccos(np.clip(1 - 2*c, -1, 1)) #Clipping to ensure we stay in the valid range
    
    return params
    
def dipole_likelihood(params):
    N_bar, D, l, b = params
    pixel_vec = np.vstack(hp.pix2vec(NSIDE, np.arange(NPIX))).T
    dipole_vec = hp.ang2vec(b, l) # dipole vector in cartesian
    angles = pixel_angles(pixel_vec, dipole_vec)
    
    lambda_i = N_bar * (1 + D*np.cos(angles))
    
    return np.sum(poisson.logpmf(m, mu=lambda_i))

# Quadrupole
def quadrupole_prior_transform(cube):
    
    params = cube.copy()
    params[0] = cube[0]*200 # 0 - 2000 (Nbar)
    params[1] = cube[1]/5 # 0 - 0.2 (Q)
    params[2] = cube[2]*2*np.pi # 0 - 2pi (l1)
    
    c = cube[3] # 0 - 1, used to calculate theta below. 
    params[3] = np.arccos(np.clip(1 - 2*c, -1, 1)) #(b1)
    
    params[4] = cube[4]*2*np.pi # 0 - 2pi (l2)
    params[5] = np.arccos(np.clip(1 - 2*c, -1, 1)) #(b2)
    
    return params
    
def quadrupole_likelihood(params):
    N_bar, Q, l1, b1, l2, b2 = params
    
    pixels = hp.pix2vec(NSIDE, np.arange(NPIX))
    
    a = hp.ang2vec(b1, l1)
    b = hp.ang2vec(b2, l2)
    
    Q_prime = np.outer(a, b)
    Q_star = 1/2 * (Q_prime + Q_prime.T)
    Q_hat = Q_star - np.trace(Q_star)/3

    f = Q*np.einsum('ij,i...,j...', Q_hat, pixels, pixels)
    
    lambda_i = N_bar * (1 + f)
    
    return np.sum(poisson.logpmf(m, mu=lambda_i))

# Dipole + Quadrupole
def dipole_quad_prior(cube):
    params = cube.copy()
    params[0] = cube[0]*200 # 0 - 200 (Nbar)
    params[1] = cube[1]/10 # 0 - 0.1 (D)
    params[2] = cube[2]/5 # 0 - 0.2 (Q)
    
    params[3] = cube[3]*2*np.pi # 0 - 2pi (l)
    c = cube[4] # 0 - 1, used to calculate theta below. 
    params[4] = np.arccos(np.clip(1 - 2*c, -1, 1)) #(b)
    
    params[5] = cube[5]*2*np.pi # 0 - 2pi (l1)
    c = cube[6] # 0 - 1, used to calculate theta below. 
    params[6] = np.arccos(np.clip(1 - 2*c, -1, 1)) #(b1)
    
    params[7] = cube[7]*2*np.pi # 0 - 2pi (l2)
    c = cube[8] # 0 - 1, used to calculate theta below. 
    params[8] = np.arccos(np.clip(1 - 2*c, -1, 1)) #(b2)
    return params

def dipole_quad_likelihood(params):
    N_bar, D, Q, l, b, l1, b1, l2, b2 = params

    dipole_vec = hp.ang2vec(b, l)
    pixels = hp.pix2vec(NSIDE, np.arange(NPIX))
    pixel_vec = np.vstack(hp.pix2vec(NSIDE, np.arange(NPIX))).T
    angles = pixel_angles(pixel_vec, dipole_vec)
    
    a = hp.ang2vec(b1, l1)
    b = hp.ang2vec(b2, l2)
    
    Q_prime = np.outer(a, b)
    Q_star = 1/2 * (Q_prime + Q_prime.T)
    Q_hat = Q_star - np.trace(Q_star)/3

    f = Q*np.einsum('ij,i...,j...', Q_hat, pixels, pixels)
    
    lambda_i = N_bar * (1 + D*np.cos(angles) + f)
    
    return np.sum(poisson.logpmf(m, mu=lambda_i))


In [ ]:
def fit_models(Nbar_list, D_list, Q_list, pathname):
# perform model 0 fitting (monopole)
    start = time.time()
    mono_sampler = ultranest.ReactiveNestedSampler(mono_param_names, monopole_likelihood, monopole_prior_transform,
                    log_dir=pathname+f'monopole_model/monopole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}',
                                                    resume='resume-similar')
    mono_result = mono_sampler.run()
    end = time.time()
    #save the time taken to fit the model   
    with open(pathname+f'monopole_model/times/monopole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}_time.txt', 'w') as f:
        f.write(f'{end - start}')

#perform model 1 fitting (dipole)
    start = time.time()
    dipole_sampler = ultranest.ReactiveNestedSampler(dipole_param_names, dipole_likelihood, dipole_prior_transform,
                    log_dir=pathname+f'dipole_model/dipole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}',
                                                    resume='resume-similar')
    dipole_result = dipole_sampler.run()
    end = time.time()

    #save the time taken to fit the model
    with open(pathname+f'dipole_model/times/dipole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}_time.txt', 'w') as f:
        f.write(f'{end - start}')
    
#perform model 2 fitting (quadrupole)
    start = time.time()
    quad_sampler = ultranest.ReactiveNestedSampler(quad_param_names, quadrupole_likelihood, quadrupole_prior_transform, 
        log_dir=pathname+f'quadrupole_model/quadrupole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}',
                                                    resume='resume-similar')

    quad_sampler.stepsampler = ultranest.stepsampler.SliceSampler(
        nsteps=10,
        generate_direction = (ultranest.stepsampler.generate_mixture_random_direction))

    quad_result = quad_sampler.run()
    end = time.time()

    #save the time taken to fit the model
    with open(pathname+f'quadrupole_model/times/quadrupole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}_time.txt', 'w') as f:
        f.write(f'{end - start}')
    
#perform model 3 fitting (dipole and quadrupole)
    start = time.time()
    dipole_quad_sampler = ultranest.ReactiveNestedSampler(dipole_quad_param_names, dipole_quad_likelihood, dipole_quad_prior,
            log_dir=pathname+f'dipole_quadrupole_model/dipole_quadrupole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}',
                                                    resume='resume-similar')

    dipole_quad_sampler.stepsampler = ultranest.stepsampler.SliceSampler(
        nsteps=10,
        generate_direction = (ultranest.stepsampler.generate_mixture_random_direction))

    dipole_quad_result = dipole_quad_sampler.run()
    end = time.time()

    #save the time taken to fit the model
    with open(pathname+f'dipole_quadrupole_model/times/dipole_quadrupole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}_time.txt', 'w') as f:
        f.write(f'{end - start}')    

    #perform model 4 fitting (to come)
    
    print(f'Finished: Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true} \n')

#         break
    
    return

In [ ]:
# Define the parameters to loop over
Nbar_list = [0.1,0.5,1,5,10,25,50,75,100,150]
D_list = [0.005,0.01,0.02,0.05,0.075,0.1]
Q_list = [0.005,0.01,0.02,0.05,0.075,0.1,0.12,0.15,0.175,0.2]


In [ ]:
pathname='../log_dir/Dipole_and_Quadrupole_Data/'

mono_param_names = [r'$\bar N$']
dipole_param_names = [r'$\bar N$', 'D', r'$\ell$', 'b']
quad_param_names = [r'$\bar N$', 'Q', r'$\ell_1$', r'$b_1$', r'$\ell_2$', r'$b_2$']
dipole_quad_param_names = [r'$\bar N$', 'D', 'Q', r'$\ell$', r'$b$', r'$\ell_1$', r'$b_1$', r'$\ell_2$', r'$b_2$']
param_combinations = itertools.product(Nbar_list, D_list, Q_list)

for N_bar_true, D_true, Q_true in param_combinations:
    # First load the data and metadata
    data = np.load(pathname+f'Datasets/Raw_files/skymap_data_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}.npy',
                    allow_pickle=True).item()
    m = data['m']
    metadata = data['metadata']
    NSIDE = metadata['NSIDE']
    NPIX = metadata['NPIX']
    
    fit_models(Nbar_list, D_list, Q_list, pathname)
    del data, m, metadata
    gc.collect()

In [ ]:
# N_bar_tester = []
# true_Nbars = []


# # loop over files
# for N_bar_true, D_true, Q_true in param_combinations:
#     with open(pathname+f'dipole_model/dipole_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}/info/results.json', 
#             'r') as f:
#         json_results = json.load(f)
#         Nbar_val = json_results['posterior']['mean'][0]
#         N_bar_tester.append(Nbar_val)
#         true_Nbars.append(N_bar_true)

# paramnames = json_results['paramnames']

# results = {
#     'paramnames': paramnames,
#     'weighted_samples': {
#         'points': samples,
#         'weights': np.ones(len(samples)) / len(samples)
#     }
# }

# cornerplot(results)
# plt.show()